# Function interpolation: Lagrange polynomial and cubic spline.

In this notebook i will implement functions to interpolate given function using Lagrange polynomial and cubic spline.

Consider following:\
    function values $f(x_i)$ are known only for some points of the grid $\omega = \{a \le x_0 < x_1 < \dots <x_N \le b\} $

Our goal is to find function $P_n(x)$ which satisfies the following conditions:
\begin{equation}\tag{1}\label{goal}
    P_n(x_i) = f_i, \; i\in [0,n], \text{ where } f_i = f(x_i)
\end{equation}

This notebook will divided into 2 big parts.

## 1. Lagrange polynomial

The interpolation polynomial in the Lagrange form is constructed using the Lagrange basis:


\begin{equation}
    l_i(x) = \dfrac{\prod_{j=0, j\ne i}^n (x-x_j)}{\prod_{j=0, j\ne i}^n (x_i-x_j)} \; i = \overline{0,n}
\end{equation}

This basis has useful feature:
\begin{equation}
    l_i(x) = \begin{cases}
                0, \; j \ne i \\
                1, \; j = i
             \end{cases}
\end{equation}
Which we will use to write polynomial:

\begin{equation}\tag{2}\label{poly:long_lagr}
    \begin{split}
        P_n(x) = \sum_{i=0}^n f_i l_i(x) = \sum_{i=0}^n f(x_i) \dfrac{\prod_{j=0, j\ne i}^n (x-x_j)}{\prod_{j=0, j\ne i}^n (x_i-x_j)} = \sum_{i=0}^n f_i \prod_{j=0, j \ne i}^n \dfrac{x-x_j}{x_i-x_j} = \\
        = \sum_{i=0}^n f_i \dfrac{(x-x_0)(x-x_1)\dots (x-x_{i-1})(x-x_{i+1}) \dots (x-x_n)}
                               {(x_i-x_0)(x_i-x_1)\dots (x_i-x_{i-1})(x_i-x_{i+1}) \dots (x_i-x_n)}
    \end{split}
\end{equation}
For convenience
\begin{equation}
    w(x) = \prod_{j=0}^n (x-x_j) \; \Rightarrow \;
    w^{'}(x) =\sum_{k=0}^n \prod_{j=0,j \ne k}^n (x-x_j) \; \Rightarrow \;
    w^{'}(x_i) =\prod_{j=0,j \ne i}^n (x_i-x_j)
\end{equation}

So we can rewrite \ref{poly:long_lagr}:

\begin{equation}\tag{2.1}\label{poly:short_lagr}
    P_n(x) = \sum_{i = 0}^n f_i \dfrac{w(x)}{(x-x_i)w^{'}(x_i)}
\end{equation}

Lagrange polynomial has another representation:

\begin{equation} \tag{2.2}\label{poly:det_lagrange}
    P_n(x) = (-1)
    \dfrac{
        \det
        \begin{pmatrix}
            0       & f_0       & f_1       & \cdots & f_n       \\
            x^n     & x_0^n     & x_1^n     & \cdots & x_n^n     \\
            x^{n-1} & x_0^{n-1} & x_1^{n-1} & \cdots & x_n^{n-1} \\
            \cdots  & \cdots    & \cdots    & \cdots & \cdots    \\
            1       & 1         & 1         & \cdots & 1         \\
        \end{pmatrix}
    }{
        \det
        \begin{pmatrix}
            x_0^n     & x_1^n     & \cdots & x_n^n     \\
            x_0^{n-1} & x_1^{n-1} & \cdots & x_n^{n-1} \\
            \cdots    & \cdots    & \cdots & \cdots    \\
            1         & 1         & \cdots & 1         \\
        \end{pmatrix}
    }
\end{equation}

We can transform it to calculate coefficients:
\begin{equation} \tag{3}
    a_i = (-1)^{n-i}
    \frac{
        \det
        \begin{pmatrix}
            f_0       & f_1       & \cdots  & \cdots & f_n       \\
            x_0^n     & x_1^n     & \cdots  & \cdots & x_n^n     \\
            \cdots    & \cdots    & \cdots  & \cdots & \cdots    \\
            x_0^{i-1} & x_1^{i-1} & \cdots  & \cdots & x_n^{i-1} \\
            x_0^{i+1} & x_1^{i+1} & \cdots  & \cdots & x_n^{i+1} \\
            \cdots    & \cdots    & \cdots  & \cdots & \cdots    \\
            1         & 1         & \cdots  & \cdots & 1         \\
        \end{pmatrix}
    }{
        \det
        \begin{pmatrix}
            x_0^n     & x_1^n     & \cdots & x_n^n     \\
            x_0^{n-1} & x_1^{n-1} & \cdots & x_n^{n-1} \\
            \cdots    & \cdots    & \cdots & \cdots    \\
            1         & 1         & \cdots & 1         \\
        \end{pmatrix}
    }, \; i = \overline{0,n}
\end{equation}

### Implementation

Let's begin by importing required libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Latex
from IPython.display import display

First we decide which function we will be interpolating. In my case this will be 
\begin{equation}
(x-2)^2 2^x-1
\end{equation}

In [ ]:
# function which we are interpolating
def function(x):
    return np.asarray((x - 2)**2 * 2**x -1)

Then we define functions to calculate coefficients $a_i$ from formula (3). We will break this process in 3 parts.

In [ ]:
def coef_numerator(x,f,i,n):
    """
    Parameters: x - grid nodes
                f - respective values
                i - current coef number
                n - polynomial degree
    """
    matr = []
    matr.append(f)
    for j in range(n,-1,-1):
        if j == i:
            continue
        row = []
        for k in range(n+1):
            row.append(x[k]**j)
        matr.append(row)
    return np.asarray(matr)


def coef_denominator(x,n):
    """
    Parameters: x - grid nodes
                n - polynomial degree
    """
    return np.asarray([[el ** i for el in x] for i in range(n, -1, -1)])


def lagrange_i_coef(x,f,i,n):
    num = coef_numerator(x,f,i,n)
    denom = coef_denominator(x,n)
    res = ((-1) ** (n-i)) * (np.linalg.det(num) / np.linalg.det(denom))
    return res

Now we are able to create function which returns all Lagrange polynomial coefficients for given degree `n`

In [ ]:
def lagrange_coefficients(x,f):
    n = len(x) - 1
    coefs = [lagrange_i_coef(x,f,i,n) for i in range(n+1)]
    return np.asarray(coefs)

To store them, we will create class `Polynomial` which i had already implemented in `approximation_least_squares.ipynb`. I will modify it a bit - adding extra variable `info` will help us to distinguish polynomials (e.g. cubic splines on different sections, about it later)

In [ ]:
class Polynomial:
    def __init__(self, coefficients, info=""):
        self.info = info
        self.coefficients = coefficients
        self.deg = np.size(coefficients) - 1
        
        
    def __getitem__(self, key):
        return self.coefficients[key]

    
    def __str__(self):
        result =f"P_{{{self.deg}}}^{{{self.info}}}(x) = {str(self.coefficients[0])}"
        sign = {True: "+", False: "-"}
        form = "".join([f"{sign[self.coefficients[i] > 0]}({abs(self.coefficients[i])})x^{{{i}}}" for i in range(1,self.deg+1)])
        return result+form
    
    
    def to_latex(self):
        result =f"Q_{self.deg} = {str(self.coefficients[0])}"
        return Latex(f"""\\begin{{equation*}}
                {str(self)}
                \\end{{equation*}}""")
    
    
    def calculate(self,x):
        return sum([self.coefficients[i]* x**i for i in np.arange(self.deg+1)])

### Optimal nodes grid

To minimize interpolation error, nodes from grid must be Chebyshev polynomial roots. To find this nodes on $[a, b]$ section, following formula should be used:


\begin{equation} \tag{4} \label{nodes:cheb}
    x_k = \dfrac{a+b}{2} + \dfrac{b-a}{2} \cos{ \dfrac{\pi (2k+1)}{2(n+1)} }, \; k = \overline{0,n}
\end{equation}


We will define function which takes number of nodes `n`,section  `a` `b` and calculates Chebyshev polynomial roots on $[a,b]$ section (optimal nodes grid) 

In [ ]:
def optimal_nodes(n,a,b):
    nodes = [(a+b)/2 + (b-a)/2 * np.cos((np.pi * (2*k +1))/(2 * (n + 1))) for k in range(n)]
    return np.asarray(nodes)

### Building Lagrange polynomials

Now we will build to lagrange polynomials to approximate given function. Second polynomial is build based on optimal grid. First lets prepare data.

In [ ]:
a,b = [0,1] # given section
n = 12 # polynomial degree
x = np.linspace(a,b, n+1 ,endpoint=True) # n+1 because we need k+1 nodes to build polynomial with k degree
f = function(x)

#### Lagrange polynomial

In [ ]:
lagrange_poly = Polynomial(lagrange_coefficients(x,f))
lagrange_poly.to_latex()

#### Lagrange polynomial on optimal grid

In [ ]:
x_optimal = optimal_nodes(n+1, a,b)
f_optimal = function(x_optimal)

lagrange_poly_optimal = Polynomial(lagrange_coefficients(x_optimal,f_optimal), info="*")
lagrange_poly_optimal.to_latex()

## 2. Cubic spline

Given same data as in the previous paragraph, cubic interpolational spline is defined by following terms:
1. \begin{equation}
        S_3(x) = \left\{ P_{3,i}(x) = a_i x^3 + b_i x^2 + c_i x + d_i \; | \; x \in [x_{i-1}, x_i], i = \overline{1,N} \right\}
   \end{equation}
    
2. \begin{equation}
       \begin{cases}
           P_{3,i}(x_i) = P_{3,i+1}(x_i) \\
           P_{3,i}^{'}(x_i) = P_{3,i+1}^{'}(x_i)  \\
           P_{3,i}^{''}(x_i) = P_{3,i+1}^{''}(x_i) 
       \end{cases}
        \; i = \overline{1,N-1}
   \end{equation}
   
3. \begin{equation}
       P_{3,i}(x_i) = f_i, \;  i = \overline{1,N-1}, \; P_{3,1}(x_0) = f_0
   \end{equation}

As there is $4N$ unknown conditions in total and we defined only $4N-2$, we need 2 more conditions to complete system of linear equations. There are some possible variants, in this notebook i will cover following:
1. \begin{equation}
       S_{3}^{'}(a) = P_{3,1}^{'}(a) = f_{0}^{'}, \; S_{3}^{'}(b) = P_{3,N}^{'}(b) = f_{N}^{'}
   \end{equation}
   
2. \begin{equation}
       S_{3}^{''}(a) = P_{3,1}^{''}(a) = f_{0}^{''}, \; S_{3}^{''}(b) = P_{3,N}^{''}(b) = f_{N}^{''}
   \end{equation}
   
3. \begin{equation}
       S_{3}^{''}(a) = P_{3,1}^{''}(a) = 0, \; S_{3}^{''}(b) = P_{3,N}^{''}(b) = 0
   \end{equation}